In [1]:
import math
import numpy as np
import pandas as pd

from glob import glob
from tqdm.auto import tqdm
from numba import njit
from matplotlib import pyplot as plt

import plotly.express as px

# !pip install duckdb

In [2]:
trainPath = "D:\\Project\\SourceTree\\DACON\\Data\\train.csv"
testPath = "D:\\Project\\SourceTree\\DACON\\Data\\test.csv"

In [3]:
# train = pd.read_csv(trainPath)
# train.head(), len(train)

test = pd.read_csv(testPath)
test.head(), len(test)

(             ID      F01      F02      F03   F04      F05  F06      F07  \
 0  TEST_0000000      NaN      NaN  IAGJDOH   NaN      NaN   -1  WBNHHNS   
 1  TEST_0000001  SDVLQID  VDDLZCR  DTZFPRW   1.0  KJLEQJC    3  OWLMRBR   
 2  TEST_0000002  XXQINZY  AXQDKZD  IAGJDOH  12.0  NFKKMGD    2  ILCWUTM   
 3  TEST_0000003  JCDXFYU  PILDDJU  IAGJDOH  22.0  LFPUEOV  209  HUHNWCJ   
 4  TEST_0000004  DZPQPXK  HKSCOJF      NaN   2.0  FMKTHYA    2  CFQLINN   
 
        F08      F09  ...      F30      F31      F32  F33      F34      F35  \
 0  MIGYEEG  NENNAGQ  ...  MFPUCBU  GTISJWW   1144.0  2.0      NaN  IRUDRFB   
 1  MIGYEEG  UKFZEAD  ...  NZGEZLW  GTISJWW  44364.0  0.0  ORGHLQN  IRUDRFB   
 2  LPYPUNA  NENNAGQ  ...  VHXETCF  GTISJWW     79.0  1.0  WRRYBLO  IRUDRFB   
 3  FTPHMPQ  FPCZMEO  ...  NZGEZLW  KHZNEZF     44.0  9.0  QMOULXS  IRUDRFB   
 4  FTPHMPQ  YOPLVLN  ...  FGOVFJM  WCPHYPZ      1.0  2.0  WZAQBGE  IRUDRFB   
 
    F36      F37  F38      F39  
 0  NaN  VINCJKA  0.0  AWWBMDJ  


In [ ]:
import duckdb
import random

con = duckdb.connect()

len_train = 28605390
chunk_size = 30000
current_chunk = 0 

# 선택된 인덱스를 추적하기 위한 집합 초기화
selected_indices = set()

# CSV 파일을 로드하고 임시 테이블을 생성
con.execute(f"""
CREATE OR REPLACE TEMP TABLE train AS
SELECT *, row_number() OVER () as idx
FROM read_csv_auto('{trainPath}')
""")

def load_random_chunk(con, path, chunk_size, total_data, selected_indices):
    # 전체 인덱스 범위에서 선택되지 않은 인덱스를 구함
    available_indices = list(set(range(total_data)) - selected_indices)

    # 선택되지 않은 인덱스에서 랜덤으로 chunk_size 만큼 샘플링
    sampled_indices = random.sample(available_indices, chunk_size)

    # sampling된 인덱스를 selected_indices에 추가
    selected_indices.update(sampled_indices)
    
    # query
    query = f"""
    SELECT *
    FROM train
    WHERE idx IN ({', '.join(map(str, sampled_indices))})
    """
    return con.query(query).df()

# 필요에 따라 반복하여 원하는 수만큼 샘플링 
train_chunks_list = list()
n = int(len_train / chunk_size) # 953 정도

n = 10 ########### 임시
for i in range(n):
    train_chunk = load_random_chunk(con, trainPath, chunk_size, len_train, selected_indices)
    train_chunks_list.append(train_chunk)

In [5]:
import pandas as pd
import category_encoders as ce

# 전체 데이터를 하나로 결합하여 인코더 피팅
combined_train_data = pd.concat(train_chunks_list, ignore_index=True)

# 범주형 변수의 목록을 가져오기
encoding_target = list(combined_train_data.dtypes[combined_train_data.dtypes == "object"].index)

# fitting of CountEncoder
enc = ce.CountEncoder(cols=encoding_target).fit(combined_train_data)

# 각 세트에 대해 동일한 encoding 적용
encoded_train_set_list = [enc.transform(train_set) for train_set in train_chunks_list]

In [8]:
train.columns

Index(['ID', 'Click', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18',
       'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38',
       'F39', 'idx'],
      dtype='object')

In [12]:
columns_set = ['ID', 'Click', 
                              'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 
                              'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 
                              'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 
                              'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39']

In [13]:
train = encoded_train_set_list[0]

# Select x, y
train_x = train.drop(columns=columns_set)
train_y = train['Click']


In [15]:
# Fill NaN
for col in train_x.columns:
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        # test_x[col].fillna(0, inplace=True)


## Adaboost

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

#---------------------------------#
#---# AdaBoostClassifier 성능 #---#
#---------------------------------#

model = AdaBoostClassifier(n_estimators=100, random_state=42)
model.fit(train_x, train_y)


for i in range(len(encoded_train_set_list)):
    test = encoded_train_set_list[i]
    test_x = test.drop(columns=columns_set)
    test_y = test['Click']

    for col in train_x.columns:
        if test_x[col].isnull().sum() != 0:
            test_x[col].fillna(0, inplace=True)
    
    pred_y = model.predict(test_x)

    # 성능 평가
    accuracy = accuracy_score(test_y, pred_y)
    print(f"{i}th Accuracy: {accuracy:.4f}")

0th Accuracy: 0.8029
1th Accuracy: 0.8050
2th Accuracy: 0.8063
3th Accuracy: 0.8037
4th Accuracy: 0.8047
5th Accuracy: 0.8040
6th Accuracy: 0.8057
7th Accuracy: 0.8060
8th Accuracy: 0.8035
9th Accuracy: 0.8045


## RandomForest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

#---------------------------------#
#---# AdaBoostClassifier 성능 #---#
#---------------------------------#

# Fill NaN
for col in train_x.columns:
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        # test_x[col].fillna(0, inplace=True)

model = RandomForestClassifier(n_estimators=1000, max_depth=18, random_state=0)
model.fit(train_x, train_y)


for i in range(len(encoded_train_set_list)):
    test = encoded_train_set_list[i]
    test_x = test.drop(columns=columns_set)
    test_y = test['Click']

    for col in train_x.columns:
        if test_x[col].isnull().sum() != 0:
            test_x[col].fillna(0, inplace=True)
    
    pred_y = model.predict(test_x)

    # 성능 평가
    accuracy = accuracy_score(test_y, pred_y)
    print(f"{i}th Accuracy: {accuracy:.4f}")

0th Accuracy: 0.8065
1th Accuracy: 0.8027
2th Accuracy: 0.8049
3th Accuracy: 0.8016
4th Accuracy: 0.8029
5th Accuracy: 0.8021
6th Accuracy: 0.8040
7th Accuracy: 0.8037
8th Accuracy: 0.8013
9th Accuracy: 0.8024


## Support Vector Machine

In [28]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix

#---------------------------------#
#---# AdaBoostClassifier 성능 #---#
#---------------------------------#

# Fill NaN
for col in train_x.columns:
    if train_x[col].isnull().sum() != 0:
        train_x[col].fillna(0, inplace=True)
        # test_x[col].fillna(0, inplace=True)

model = svm.SVC(decision_function_shape='ovo')
model.fit(train_x, train_y)


for i in range(len(encoded_train_set_list)):
    test = encoded_train_set_list[i]
    test_x = test.drop(columns=columns_set)
    test_y = test['Click']

    for col in train_x.columns:
        if test_x[col].isnull().sum() != 0:
            test_x[col].fillna(0, inplace=True)
    
    pred_y = model.predict(test_x)

    # 성능 평가
    accuracy = accuracy_score(test_y, pred_y)
    print(f"{i}th Accuracy: {accuracy:.4f}")

0th Accuracy: 0.8027
1th Accuracy: 0.8053
2th Accuracy: 0.8067
3th Accuracy: 0.8040
4th Accuracy: 0.8049
5th Accuracy: 0.8041
6th Accuracy: 0.8059
7th Accuracy: 0.8062
8th Accuracy: 0.8036
9th Accuracy: 0.8047
